<a href="https://colab.research.google.com/github/barbaroja2000/rubric/blob/main/Rubric_Anthropic_Claude_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluating Anthropic Claude 2.1 against a custom Rubrik with GPT-4

Uses:

*   Langsmith for custom rubrik datasets & Evaluation Framework
*   GPT-4 To evaluate LLM output

---

Test:

*  provides a quality assesment 0-5 and a rationalle for the score

Model Anthropic Claude 2.1:

* 200K Context Window: Doubling the information limit to 200,000 tokens, Claude 2.1 can handle large documents like entire codebases and long literary works, enabling functionalities like summarization and trend forecasting.
* Improved Honesty and Comprehension: Significant reduction in false statements, with a 2x decrease compared to Claude 2.0, and better performance in comprehending and summarizing complex documents.

https://www.anthropic.com/index/claude-2-1

https://python.langchain.com/docs/langsmith/


---

Requires:
*   Test dataset in Langsmith: This notebook shows how to set this up https://github.com/barbaroja2000/rubric/blob/main/Rubric_Langsmith_Dataset_Creator.ipynb
*   Anthropic API Key
*   Langchain Key
*   OpenAI Key





In [1]:
!pip install -q langchain langsmith anthropic openai tiktoken

In [2]:
from google.colab import userdata
import os
import requests

os.environ['LANGCHAIN_ENDPOINT']= "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']= userdata.get('langchain_api_key')
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ['ANTHROPIC_API_KEY']= userdata.get('anthropic_api_key')

## Run Metadata

In [3]:
project_name  = "Anthropic Claude 2.1"
rubric_model = "claude-2.1"
rubric_temperature=0
evaluator_model="gpt-4-1106-preview"
evaluator_temperature=0
dataset_name = "General Rubric: Extended"

In [4]:
from langsmith import Client
client = Client()

In [5]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# create a string template for a System role
system_template = """
Please provide a detailed, accurate, and contextually relevant response to the following query.
Your answer should demonstrate a clear understanding of the subject matter, considering any logical,
ethical, historical, or cultural aspects involved. Ensure the response is tailored to the specific
requirements of the question, whether it involves analysis, explanation, creativity, or problem-solving.
"""

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.chat_models import ChatAnthropic


def create_runnable():
    llm = ChatAnthropic(model=rubric_model, temperature=rubric_temperature)
    prompt = ChatPromptTemplate.from_messages([
      ("system", system_template),
      ("human", "{input}")
    ])
    return RunnableMap({"input": RunnablePassthrough()}) | prompt | llm

## Evaluator

https://docs.smith.langchain.com/evaluation/evaluator-implementations

In [7]:
import re
from typing import Any, Optional

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import StringEvaluator

class QualityEvaluator(StringEvaluator):
    """An LLM-based quality  evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model=evaluator_model, temperature=evaluator_temperature)

        template = """ Evaluate the answer provided from 0 to 5, with 5 being the best quality answer, and 0 where the answer is false or not provided
        --------
        QUESTION: {input}
        --------
        ANSWER: {prediction}
        --------
        Consisely reason step by step about why the given score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = LLMChain.from_string(llm=llm, template=template)

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "quality"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain(
            dict(input=input, prediction=prediction), **kwargs
        )
        lines = evaluator_result["text"].strip().split("\n")
        reasoning = evaluator_result["text"].strip()
        score = lines[-1]
        if score is not None:
            score = float(score.strip())
        return {"score": score, "reasoning": reasoning}

In [8]:
import random
import string

def rando():
  # Generating a random 4-letter string
  random_string = ''.join(random.choices(string.ascii_letters, k=4))
  return random_string

## Test Run

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    input_key="question",
    output_key="output",
    custom_evaluators = [QualityEvaluator()],
)

project_metadata = {
  rubric_model : rubric_model,
  rubric_temperature : rubric_temperature,
  evaluator_model : evaluator_model,
  evaluator_temperature: evaluator_temperature
}

run = run_on_dataset(
    client=client,
    project_metadata = project_metadata,
    project_name = f"{project_name}-{rando()}",
    dataset_name=dataset_name,
    llm_or_chain_factory=create_runnable,
    evaluation=evaluation_config,
    concurrency_level=1, #This will bork if not provided or set to anything other than 1 for Anthropic without $$ account
)

View the evaluation results for project 'Anthropic Claude 2.1-SnGi' at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd/compare?selectedSessions=cb91ab1d-cb07-49fa-829f-26c6432dd928

View all tests for Dataset General Rubric: Extended at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd
[--------------------------------------->          ] 99/123

In [ ]:
# Extracting the required information
extracted_data = []
for key, value in run['results'].items():
    output = value['output'].content
    question = value['input']['question']
    for feedback_item in value['feedback']:
        if feedback_item.key == 'quality':
            quality = feedback_item.score
            feedback = feedback_item.comment
            extracted_data.append({
                'question': question,
                'output': output,
                'quality': quality,
                'feedback': feedback
            })

In [19]:
import pandas as pd
from google.colab import data_table
df = pd.DataFrame(extracted_data)

data_table.DataTable(df, include_index=False, num_rows_per_page=10)

,question,output,quality,feedback
0,"I WAS born in the year 1632, in the city of Yo...",Here is a 5-word summary of the passage:\n\nM...,4.0,The provided answer is a concise summary of th...
1,"I WAS born in the year 1632, in the city of Yo...",Here is a bullet point summary of the key poi...,5.0,Evaluation of the Answer:\n\n- The answer prov...
2,"Y curiosity, in a sense, was stronger than my ...",Here is a summary of the key themes in the pa...,4.0,The provided answer does a good job of summari...
3,Explain the role of a central bank in an econo...,Here is an explanation of the role of a centr...,4.0,The provided answer gives a clear and understa...
4,Describe the basics of how the stock market wo...,Here is a basic explanation of how the stock ...,4.0,The provided answer gives a clear and concise ...
...,...,...,...,...
118,Analyze the emotional content of this diary en...,Unfortunately there is not enough emotional c...,5.0,The answer provided is comprehensive and well-...
119,What is the emotional tone of this speech exce...,"Based on the content, this speech excerpt has...",5.0,The provided answer does a good job of analyzi...
120,Determine the emotional tone of this message: ...,"The message ""I am deeply disappointed with yo...",5.0,The answer provided is of high quality. It acc...
121,Analyze the emotion in this customer review: '...,Based on the very positive language used in t...,5.0,The provided answer does an excellent job of a...


In [18]:
pd.to_numeric(df.quality).mean()

4.203252032520325

In [22]:
examples = list(client.list_examples(dataset_name=dataset_name))

In [21]:
# Creating a new DataFrame from the list
list_df = pd.DataFrame([{'question': item.inputs['question'], 'category': item.inputs['category']} for item in examples])

In [20]:
# Merging the DataFrames
merged_df = pd.merge(df, list_df, on='question')

In [17]:
# Group by 'category' and calculate the mean of 'quality' scores
avg_scores_per_category = merged_df.groupby('category')['quality'].mean()

print(avg_scores_per_category)

category
Categorization                           4.727273
Coding                                   4.230769
Creative Writing                         2.083333
Cultural and Contextual Understanding    4.400000
Emotion Analysis                         5.000000
Ethics Analysis                          4.000000
Fact Analysis                            4.833333
Logical Reasoning                        3.818182
Reading Comprehension                    5.000000
Reframing                                4.250000
Safety and Security                      4.375000
Summarization                            4.333333
Name: quality, dtype: float64
